## Importing libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
from pandas.plotting import scatter_matrix
import math


import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

In [57]:
df = pd.read_csv('clean_data_for_regression.csv',index_col=0)

In [58]:
df.head()

,American Indian/Alaska Native,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander,White,grade,male,preschool,...,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
0,0,0,0,0,0,0,1,9,1,1.0,...,1.0,1.0,0.0,250.0,28.0,1.0,1,0,1233.0,554.81
1,0,0,1,0,0,0,0,10,0,1.0,...,1.0,1.0,1.0,200.0,23.0,1.0,1,1,2640.0,458.11
2,0,0,0,1,0,0,0,10,1,1.0,...,1.0,1.0,1.0,250.0,35.0,1.0,1,1,1095.0,613.89
3,0,0,0,0,0,0,1,10,0,1.0,...,1.0,1.0,1.0,300.0,30.0,1.0,1,0,1913.0,439.36
4,0,0,0,0,0,0,1,9,0,1.0,...,1.0,1.0,0.0,378.0,20.0,0.0,1,1,502.0,428.96


Splitting the data into training and testing sections

In [62]:
target = df['readingScore']
features = df[['American Indian/Alaska Native', 'Asian', 'Black', 'Hispanic',
       'More than one race', 'Native Hawaiian/Other Pacific Islander', 'White',
       'grade', 'male', 'preschool', 'expectBachelors', 'motherHS',
       'motherWork', 'fatherHS', 'fatherWork', 'selfBornUS', 'motherBornUS',
       'fatherBornUS', 'englishAtHome', 'computerForSchoolwork',
       'read30MinsADay', 'minutesPerWeekEnglish', 'studentsInEnglish',
       'schoolHasLibrary', 'publicSchool', 'urban', 'schoolSize']]

In [82]:
x_train, x_test, y_train, y_test = train_test_split(features,target,test_size=.2,random_state=23)

In [83]:
x_training, x_val, y_training, y_val = train_test_split(x_train,y_train,test_size=.2,random_state=23)

### Scaling down our continuous data

In [84]:
scaler = StandardScaler()
continuous = ['minutesPerWeekEnglish','studentsInEnglish', 'schoolSize']
for var in continuous:
    scaler.fit(x_training[[var]])
    x_training[var+'_scaled'] = scaler.transform(x_training[[var]])
    x_val[var+'_scaled'] = scaler.transform(x_val[[var]])
    x_test[var+'_scaled'] = scaler.transform(x_test[[var]])

x_training.drop(columns=continuous,inplace=True)
x_val.drop(columns=continuous,inplace=True)
x_test.drop(columns=continuous,inplace=True)

/Users/mendeloster/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mendeloster/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mendeloster/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

Introducing all our features to an OLS model

In [95]:
ols_formula = 'readingScore~Asian+Black+Hispanic+White+grade+male+preschool+expectBachelors+motherHS+motherWork+fatherHS+fatherWork+selfBornUS+motherBornUS+fatherBornUS+englishAtHome+computerForSchoolwork+read30MinsADay+minutesPerWeekEnglish_scaled+studentsInEnglish_scaled+schoolHasLibrary+publicSchool+urban+schoolSize_scaled'
ols_model = ols(ols_formula,data=pd.concat([x_training,y_training],axis=1)).fit()

In [97]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           readingScore   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     31.96
Date:                Thu, 26 Mar 2020   Prob (F-statistic):          4.40e-122
Time:                        11:29:00   Log-Likelihood:                -11410.
No. Observations:                1991   AIC:                         2.287e+04
Df Residuals:                    1966   BIC:                         2.301e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      171.6509     39.221      4.377      0.000      94.732     248.570
Asian                           30.6617     11.894      2.578      0.010       7.335      53.988
Black                          -35.5856      9.976     -3.567      0.000     -55.150     -16.021
Hispanic                        -8.6054      9.177     -0.938      0.349     -26.604       9.393
White                           28.7927      8.229      3.499      0.000      12.655      44.930
grade                           26.3487      3.358      7.846      0.000      19.763      32.935
male                           -10.9314      3.487     -3.135      0.002     -17.769      -4.094
preschool                        1.5502      3.933      0.394      0.694      -6.163       9.263
expectBachelors                 60.9353      4.658     13.082      0.000      51.800      70.071
motherHS                         7.8977      6.950      1.136      0.256      -5.732      21.527
motherWork                       2.0641      3.903      0.529      0.597      -5.590       9.719
fatherHS                        10.5805      6.333      1.671      0.095      -1.839      23.000
fatherWork                       4.6825      4.908      0.954      0.340      -4.943      14.308
selfBornUS                      -0.1487      8.214     -0.018      0.986     -16.258      15.960
motherBornUS                    -5.3262      7.490     -0.711      0.477     -20.016       9.364
fatherBornUS                     4.1652      7.198      0.579      0.563      -9.951      18.281
englishAtHome                   13.7239      7.833      1.752      0.080      -1.639      29.087
computerForSchoolwork           21.4903      6.935      3.099      0.002       7.890      35.091
read30MinsADay                  37.2584      3.809      9.781      0.000      29.788      44.729
minutesPerWeekEnglish_scaled     0.9977      1.753      0.569      0.569      -2.440       4.435
studentsInEnglish_scaled        -3.3352      1.859     -1.794      0.073      -6.981       0.310
schoolHasLibrary               -18.3318      9.755     -1.879      0.060     -37.463       0.799
publicSchool                   -23.2886      6.959     -3.346      0.001     -36.937      -9.640
urban                           -5.3494      4.454     -1.201      0.230     -14.085       3.386
schoolSize_scaled               11.0214      2.137      5.156      0.000       6.829      15.213
==============================================================================
Omnibus:                        3.793   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.150   Jarque-Bera (JB):                3.803
Skew:                          -0.090   Prob(JB):                        0.149
Kurtosis:                       2.883   Cond. No.  